# Add extra information to DRZ auction results

Query to the open data dataset of the RDW.

It may take a while (10 min) to query all auction results.
- - - - 

### User variables


In [1]:
Date = '2019-03' # yyyy-mm

### Modules and functions

In [2]:
import pandas as pd

# to keep api key hidden import this from sub dir
import assets.hidden_api_keys as hidden_api_keys

# base url
apiurl = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json?$$app_token=' + hidden_api_keys.socrata_apptoken + '&'

def get_json_from_api(url,reg,c=0):
    
    '''Get json object from api'''
    
    import time

    c+=1
    try:
        df=pd.read_json(url + 'kenteken=' + reg.replace('-','').upper()).to_dict()
    except:
        if c > 10:
            print(url,reg)
            raise 
        else:
            print('pause 2 sec and try again!')
            time.sleep(2)
            df = get_json_from_api(url,reg,c)
    
    return df
    
# get_json_from_api(apiurl,'61-sf-FG')

### Load auction results

In [3]:
file_name = '../data/drz-data-{}.pkl'.format(Date)
print(file_name)
drz = pd.read_pickle(file_name)

../data/drz-data-2019-03.pkl


### query rdw

In [4]:
# see what lots have a Dutch registration (license number).
hasReg = (~drz.Reg.isnull()) & (drz.Reg != 'onbekend') & (drz.Reg != '') & (~drz.LotType.isin([
    'Vaartuig',
    'Jetski',
    'Sloep',
    'Speedboot',
    'Vaartuig (Type onbekend)',
    'Motorvaartuig met opbouw (Pleziervaartuig)',
    'Aanhangwagen',
]))

# make a copy and add info
rdw = drz.copy()
rdw["rdwinfo"]=None

In [5]:
def get_query_url(api_url, regs, token=hidden_api_keys.socrata_apptoken):
    '''construct query url'''
    
    # convert list to string
    reg_list = ''.join(["'{}', ".format(r.replace('-','').upper()) for r in regs])
    reg_list = reg_list[0:-2] # remove trailing ', '
    q = api_url + '?$$app_token=' + token + '&$where='
    # add escaped soql
    q += urllib.parse.quote('kenteken in(' + reg_list + ')')
    return q

In [6]:
import urllib

# do main api first to get other possible apis
api_name = 'api_gekentekende_voertuigen'
api_url = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json'
regs = rdw.loc[hasReg,'Reg'].values

# query data base
dfs = list()
q = get_query_url(api_url,regs)
dfs.append(pd.read_json(q))
dfs[0].set_index('kenteken', inplace=True)
dfs[0].tail()

# query other available apis
for api_name in [c for c in dfs[0].columns if c.startswith('api')]:
    print(api_name)
    for api_url in dfs[0][api_name].unique():
        print(api_url)
        q = get_query_url(api_url,regs)
        df0 = pd.read_json(q)
        df0.columns.name = api_name
        if df0.shape[0] != 0:
            df0.set_index('kenteken', inplace=True)
        if api_name == 'api_gekentekende_voertuigen_assen':
            df0 = pd.pivot(df0, columns='as_nummer')
            df0 = df0.reorder_levels([1,0], axis='columns').sort_index(axis='columns', level=0)

        elif api_name == 'api_gekentekende_voertuigen_brandstof':
            df0 = pd.pivot(df0, columns='brandstof_volgnummer')
            df0 = df0.reorder_levels([1,0], axis='columns').sort_index(axis='columns', level=0)

        elif api_name == 'api_gekentekende_voertuigen_carrosserie':
            df0 = pd.pivot(df0, columns='carrosserie_volgnummer')
            df0 = df0.reorder_levels([1,0], axis='columns').sort_index(axis='columns', level=0)
        elif api_name == 'api_gekentekende_voertuigen_carrosserie_specifiek':
            df0 = pd.pivot(df0, columns='carrosserie_volgnummer')
            df0 = df0.reorder_levels([1,0], axis='columns').sort_index(axis='columns', level=0)

        display(df0.tail())
        dfs.append(df0)
        
        
df = pd.concat(dfs, axis='columns', sort=False)


api_gekentekende_voertuigen_assen
https://opendata.rdw.nl/resource/3huj-srit.json


as_nummer                                      1                     \
api_gekentekende_voertuigen_assen aangedreven_as aantal_assen hefas   
kenteken                                                              
TZBT99                                       NaN          2.0   NaN   
V718FB                                       NaN          2.0     N   
VG420S                                         J          2.0     N   
XG037X                                       NaN          2.0   NaN   
XPJJ71                                       NaN          2.0   NaN   

as_nummer                                                     \
api_gekentekende_voertuigen_assen plaatscode_as spoorbreedte   
kenteken                                                       
TZBT99                                      NaN        137.0   
V718FB                                        V        174.0   
VG420S                                        V        151.0   
XG037X                                        V        150.0   
XPJJ71                                      NaN        137.0   

as_nummer                                                              \
api_gekentekende_voertuigen_assen technisch_toegestane_maximum_aslast   
kenteken                                                                
TZBT99                                                          740.0   
V718FB                                                         1600.0   
VG420S                                                            NaN   
XG037X                                                          905.0   
XPJJ71                                                          625.0   

as_nummer                                                              \
api_gekentekende_voertuigen_assen wettelijk_toegestane_maximum_aslast   
kenteken                                                                
TZBT99                                                          740.0   
V718FB                                                         1600.0   
VG420S                                                         1080.0   
XG037X                                                          905.0   
XPJJ71                                                          625.0   

as_nummer                                      2                     \
api_gekentekende_voertuigen_assen aangedreven_as aantal_assen hefas   
kenteken                                                              
TZBT99                                       NaN          2.0   NaN   
V718FB                                       NaN          2.0     N   
VG420S                                         N          2.0     N   
XG037X                                       NaN          2.0   NaN   
XPJJ71                                       NaN          2.0   NaN   

as_nummer                                                     \
api_gekentekende_voertuigen_assen plaatscode_as spoorbreedte   
kenteken                                                       
TZBT99                                      NaN        140.0   
V718FB                                        A        173.0   
VG420S                                        A        156.0   
XG037X                                        A        151.0   
XPJJ71                                      NaN        134.0   

as_nummer                                                              \
api_gekentekende_voertuigen_assen technisch_toegestane_maximum_aslast   
kenteken                                                                
TZBT99                                                          730.0   
V718FB                                                         1760.0   
VG420S                                                            NaN   
XG037X                                                         1045.0   
XPJJ71                                                          700.0   

as_nummer                                                              
api_geken

api_gekentekende_voertuigen_brandstof
https://opendata.rdw.nl/resource/8ys7-d773.json


brandstof_volgnummer                                       1  \
api_gekentekende_voertuigen_brandstof brandstof_omschrijving   
kenteken                                                       
V718FB                                                Diesel   
VG420S                                                Diesel   
VZBL90                                                Diesel   
XG037X                                               Benzine   
XPJJ71                                               Benzine   

brandstof_volgnummer                                            \
api_gekentekende_voertuigen_brandstof brandstofverbruik_buiten   
kenteken                                                         
V718FB                                                     NaN   
VG420S                                                     4.4   
VZBL90                                                     NaN   
XG037X                                                     6.2   
XPJJ71                                                     4.7   

brandstof_volgnummer                                                  \
api_gekentekende_voertuigen_brandstof brandstofverbruik_gecombineerd   
kenteken                                                               
V718FB                                                           NaN   
VG420S                                                           4.8   
VZBL90                                                           NaN   
XG037X                                                           8.4   
XPJJ71                                                           5.5   

brandstof_volgnummer                                          \
api_gekentekende_voertuigen_brandstof brandstofverbruik_stad   
kenteken                                                       
V718FB                                                   NaN   
VG420S                                                   5.4   
VZBL90                                                   NaN   
XG037X                                                  12.1   
XPJJ71                                                   6.9   

brandstof_volgnummer                                             \
api_gekentekende_voertuigen_brandstof co2_uitstoot_gecombineerd   
kenteken                                                          
V718FB                                                    221.0   
VG420S                                                    125.0   
VZBL90                                                      NaN   
XG037X                                                    203.0   
XPJJ71                                                    130.0   

brandstof_volgnummer                                        \
api_gekentekende_voertuigen_brandstof co2_uitstoot_gewogen   
kenteken                                                     
V718FB                                                 NaN   
VG420S                                                 NaN   
VZBL90                                                 NaN   
XG037X                                                 NaN   
XPJJ71                                                 NaN   

brandstof_volgnummer                                            \
api_gekentekende_voertuigen_brandstof emissiecode_omschrijving   
kenteken                                                         
V718FB                                                  Euro 4   
VG420S                                                  Euro 5   
VZBL90                                                     NaN   
XG037X                                                  Euro 4   
XPJJ71                                                  Euro 2   

brandstof_volgnummer                                         \
api_gekentekende_voertuigen_brandstof geluidsniveau_rijdend   
kenteken                                                      
V718FB                                                  NaN   
VG420S                                               

api_gekentekende_voertuigen_carrosserie
https://opendata.rdw.nl/resource/vezc-m2t6.json


carrosserie_volgnummer                                1  \
api_gekentekende_voertuigen_carrosserie carrosserietype   
kenteken                                                  
V718FB                                               BB   
VG420S                                               BB   
VZBL90                                               BA   
XG037X                                               AD   
XPJJ71                                               AB   

carrosserie_volgnummer                                                          
api_gekentekende_voertuigen_carrosserie type_carrosserie_europese_omschrijving  
kenteken                                                                        
V718FB                                                             Bestelwagen  
VG420S                                                             Bestelwagen  
VZBL90                                                             Vrachtwagen  
XG037X                                                                   Coupe  
XPJJ71                                                               Hatchback

api_gekentekende_voertuigen_carrosserie_specifiek
https://opendata.rdw.nl/resource/jhie-znh9.json


carrosserie_volgnummer                                                                      1  \
api_gekentekende_voertuigen_carrosserie_specifiek carrosserie_voertuig_nummer_code_volgnummer   
kenteken                                                                                        
99WHJX                                                                                      1   
V718FB                                                                                      1   
VG420S                                                                                      1   

carrosserie_volgnummer                                                                               \
api_gekentekende_voertuigen_carrosserie_specifiek carrosserie_voertuig_nummer_europese_omschrijving   
kenteken                                                                                              
99WHJX                                                                           Voor vervoer boten   
V718FB                                                                              Gesloten opbouw   
VG420S                                                                              Gesloten opbouw   

carrosserie_volgnummer                                             
api_gekentekende_voertuigen_carrosserie_specifiek carrosseriecode  
kenteken                                                           
99WHJX                                                         21  
V718FB                                                          3  
VG420S                                                          3

api_gekentekende_voertuigen_voertuigklasse
https://opendata.rdw.nl/resource/kmfi-hrps.json


api_gekentekende_voertuigen_voertuigklasse


In [7]:
df['TimeStamp'] = pd.to_datetime('now').strftime('%Y%m%d')


# rdw['rdwinfo'] = df


### Saving

In [ ]:
file_name = '../data/rdw-data-{}.pkl'.format(Date)
print(file_name)
rdw.to_pickle(file_name)